In [ ]:
import numpy as np
import pandas as pd
import os
import re
import networkx as nx
import pickle
import json
from numpy import genfromtxt

In [ ]:
data_dir = '/home/ubuntu/Stella/MLVU_multimodality/valid_DTI_con_mat_1105'

In [ ]:
targ_folder = os.listdir(data_dir)

In [ ]:
con_mat_list=list(targ_folder)

In [ ]:
len(con_mat_list)

In [ ]:
con_mat_list[0]

In [ ]:
node_skeleton = pd.read_csv('DTI - sMRI.csv')

In [ ]:
node_data = pd.read_csv('mor.some.qc.csv')

In [ ]:
node_data['subjectkey'] = node_data['subjectkey'].apply(lambda x : x.replace('_','')) #remove _ from subjectkey

In [ ]:
node_skeleton_array  = np.array(node_skeleton.iloc[:, 2:])

In [ ]:
node_skeleton_array.shape

In [ ]:
node_skeleton_array[9][1]

In [ ]:
data = os.path.join(data_dir, con_mat_list[0])
subject_id = con_mat_list[0].split(' ')[4].split('.')[0]

# make adjacency matrix
mydata = genfromtxt(data, delimiter=',')
adjacency = mydata[1:,1:]

In [ ]:
adjacency.shape

In [ ]:
def node_feat_func(subject_id):
    node_feat = list()
    data = node_data[node_data['subjectkey'] == subject_id]
    for i in range (84):
        col_name = node_skeleton_array[i][1]
        if pd.isna(col_name) == True:
            node_feat.append(0)
        else:
            node_feat.append(round(data[col_name].values[0]))
    # change format
    keylist=[]
    valuelist=[]

    for j,k in enumerate(node_feat):
        keylist.append(str(j))
        valuelist.append(str(k))
    node_feat_dict=dict(zip(keylist, valuelist)) 
    return node_feat_dict

In [ ]:
test = node_feat_func('NDARINVUN80YW88')

In [ ]:
test

In [ ]:
edge_list = edge_list_func(adjacency)

In [ ]:
num_edges = num_edges_func(edge_list)

In [ ]:
edge_index = edge_feat_func(num_edges, edge_list)

In [ ]:
len(edge_index)

In [ ]:
def edge_list_func(adjacency):
    edge_list=[]
    for i in range (84):
        for j in range (84):
            edge_list.append((i, j, adjacency[i][j]))
                
    edge_new_list=[]
    for i in range (len(edge_list)):
        if edge_list[i][2] >= 0.2:
            edge_new_list.append(edge_list[i])
            
    return edge_new_list

In [ ]:
def num_edges_func(edge_list):
    num_edges=len(edge_list)
    return num_edges

In [ ]:
def edge_feat_func(num_edges, edge_list):
    # initialization
    edgefeat_dim = 1
    edge_feat = np.zeros((int(num_edges), edgefeat_dim))
    edge_index = np.zeros((int(num_edges),2))
    # calculation
    for i in range (num_edges):
        source = edge_list[i][0]
        target = edge_list[i][1]
        edge_index[i][0] = int(source)
        edge_index[i][1] = int(target)
    # change format
    edge_index_list=[]
    for i in range(edge_index.shape[0]):
        source = int(edge_index[i][0])
        target = int(edge_index[i][1])
        edge_index_list.append([source, target])
    return edge_index_list

In [ ]:
for i in range(len(con_mat_list)):
    # load data
    data = os.path.join(data_dir, con_mat_list[i])
    subject_id = con_mat_list[i].split(' ')[4].split('.')[0]
    
    # make adjacency matrix
    mydata = genfromtxt(data, delimiter=',')
    adjacency = mydata[1:,1:]
    
    # edge_index
    edge_list = edge_list_func(adjacency)
    num_edges = num_edges_func(edge_list)
    edge_index = edge_feat_func(num_edges, edge_list)
    
    # node_feat
    node_feat = node_feat_func(subject_id)
    
    # draw graph
    graph = dict({'edges': edge_index, 'features': node_feat})
    
    print(graph)
    
    with open('/home/ubuntu/Stella/MLVU_multimodality/build structural graph/structural_graph_for_graph2vec_1212/graph2vec_structural_graph_%s.json'%subject_id, 'wb') as file:
        pickle.dump(graph, file)